In [1]:
import requests
from bs4 import BeautifulSoup
myBaseUrl = "https://www.amazon.com/Special-Supplies-Independence-Lightweight-Spill-Resistant/product-reviews/B077739BGX/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber="

start_urls = []
for i in range(1, 19): # for different product change second number in range to number of review pages + 1
    start_urls.append(myBaseUrl + str(i))

HEADERS = {
    'authority': 'www.amazon.com',
'method' : 'POST',
'path' : '/gp/redirection/singapore.html',
'scheme' : 'https',
'accept':'*/*',
'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
} 
response = []
for i in start_urls:
    response.append(requests.get(i,headers=HEADERS))
    print(response)

[<Response [200]>]
[<Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]
[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [2]:
soups = []
for i in response:
    soups.append(BeautifulSoup(i.content))

reviews = []
for soup in soups:
    review = soup.findAll("span",{'class':"a-size-base review-text review-text-content"})
    for i in review:
        reviews.append(i.text.strip())

In [3]:
titles = []
for soup in soups:
    title = soup.findAll("a",{'class':"a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    for i in title:
        titles.append(i.text.strip())

In [4]:
avg_rating = 4.4 
stars = []
for soup in soups:
    star = soup.findAll("span",{'class':"a-icon-alt"})
    for i in star: 
        stars.append((i.text.replace('out of 5 stars', '')).strip())

#code to remove excess stars scraped from html 
finalstars = []
indexofavg_rating = 0; 
for i in range(len(stars)):
    if str(avg_rating) in stars[i]:
        indexofavg_rating = i
    elif i != indexofavg_rating:
        if i != indexofavg_rating + 1:
            if i != indexofavg_rating + 2:
                finalstars.append(stars[i])

In [5]:
import pandas as pd

df = pd.DataFrame(titles, columns=['title'])
df.to_csv('soupindependence.csv', index=False) 
read = pd.read_csv("soupindependence.csv")
read["review"]= reviews
read.to_csv("soupindependence.csv", index=False)
read["star"]= finalstars
read.to_csv("soupindependence.csv", index=False)
df = pd.read_csv("soupindependence.csv")

In [6]:
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df['polarity'] = df['review'].apply(pol)
df['subjectivity'] = df['review'].apply(sub)
df

,title,review,star,polarity,subjectivity
0,awesome,I purchased this cup as a last resort to help ...,5.0,0.330455,0.321818
1,No more nighttime spills!,"Believe it or not, this was purchased for my 9...",5.0,0.143878,0.490816
2,NOT SPILL PROOF,Headline says it all,1.0,0.000000,0.000000
3,Great help for people with Alzheimer’s,Bought for mother who has Alzheimer’s. She ha...,5.0,0.373333,0.683333
4,No more spills yay!,This cup is sturdy and well designed. Haven't ...,5.0,0.259416,0.440867
5,Disappointed,I returned this item. Too small. More for a b...,2.0,0.000000,0.662500
6,Not spillproof,Nice non-tip cup with easy to hold 2 handles....,3.0,0.064583,0.802083
7,Light and durable,"We got this cups for grandma, she is in her 90...",5.0,0.166667,0.507051
8,Great Product!,I bought this for my son who had a stroke at 4...,5.0,0.187500,0.500000
9,Great for elderly with joint or muscle weakness.,This clear plastic mug is great use for the el...,5.0,0.450000,0.566667
